In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pyspark


In [2]:
spark = (SparkSession.builder.appName('dataframe_file')
         .config('spark.dynamicAllocation.enabled',"false")
         .config('spark.sql.adaptive.enabled',"false")
         .config("spark.executor.extraClassPath",'/home/daniel/jdbc_folder/postgresql42.7.0.jar')
         .config("spark.driver.extraClassPath", '/home/daniel/jdbc_folder/postgresql42.7.0.jar')
         .config("spark.jars.packages", "org.postgresql:postgresql:42.7.0") 
         .getOrCreate())   

your 131072x1 screen size is bogus. expect trouble
23/12/04 06:14:53 WARN Utils: Your hostname, DESKTOP-MU6A7H5 resolves to a loopback address: 127.0.1.1; using 172.25.134.246 instead (on interface eth0)
23/12/04 06:14:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/daniel/data_engineering/sparkenv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/daniel/.ivy2/cache
The jars for the packages stored in: /home/daniel/.ivy2/jars
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4b80c27b-293c-492d-9e91-34c4ed88ca7f;1.0
	confs: [default]
	found org.postgresql#postgresql;42.7.0 in central
	found org.checkerframework#checker-qual;3.31.0 in central
:: resolution report :: resolve 1613ms :: artifacts dl 43ms
	:: modules in use:
	org.checkerframework#checker-qual;3.31.0 from central in [default]
	org.postgresql#postgresql;42.7.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-----------------------------------------

In [3]:
spark

In [4]:
yellowtaxicsv = spark.read.csv('DataFiles/Raw/YellowTaxis_*.csv',header=True)

In [5]:
yellowtaxicsv.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1|2022-10-01T05:33:...| 2022-10-01T05:48:...|            1.0|          1.7|       1.0|                 N|         249|         107|           1|        9.5|  3.0|    0.5|      2.6

In [6]:
yellowtaxicsv.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- RatecodeID: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- extra: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: string (nullable = true)
 |-- congestion_surcharge: string (nullable = true)
 |-- airport_fee: string (nullable = true)



In [7]:
yellowtaxicsvde = yellowtaxicsv.describe(['passenger_count','trip_distance'])

In [8]:
yellowtaxicsvde.show()

+-------+------------------+-----------------+
|summary|   passenger_count|    trip_distance|
+-------+------------------+-----------------+
|  count|           3542399|          3675419|
|   mean| 1.384692972192009|6.206967213806036|
| stddev|0.9302299062876463|640.8230706269816|
|    min|               0.0|              0.0|
|    max|               9.0|            99.89|
+-------+------------------+-----------------+



# performing data cleansing on dataset

In [9]:

yellowTaxicleansing =( yellowtaxicsv.where('passenger_count > 0.0 ').where('trip_distance > 0.0').na.drop('all')
               .dropDuplicates().where('tpep_pickup_datetime > "2022-10-01" AND tpep_dropoff_datetime > "2022-11-01" '))


In [10]:
yellowTaxicleansing.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2|2022-11-01T00:10:...| 2022-11-01T00:22:...|            1.0|         1.41|       1.0|                 N|         239|         143|           1|        9.0|  1.0|    0.5|      2.6

In [11]:
#total number of records after cleansing
yellowTaxicleansing.count()

28394

## selecting the number of column to be used as per business logic

In [12]:
yellowTaxicleansing.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- RatecodeID: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- extra: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: string (nullable = true)
 |-- congestion_surcharge: string (nullable = true)
 |-- airport_fee: string (nullable = true)



In [13]:
selected_taxi_column = yellowTaxicleansing.select("VendorID",
                                                  col("passenger_count").cast('integer'),
                                                  col("store_and_fwd_flag").alias("flag"),
                                                 'PULocationID',
                                                 'DOLocationID',
                                                 'payment_type',
                                                 'fare_amount',
                                                 'extra',
                                                 'total_amount',
                                                'tip_amount',
                                                  col('tpep_dropoff_datetime').alias('dropoff_time'),
                                                  col('tpep_pickup_datetime').alias('pickup_time')  )

# renaming few columns

In [14]:
selected_taxi_column = selected_taxi_column.withColumnRenamed('PULocationID','pickupLocationID').withColumnRenamed('DOLocationID','dropoffLocationID')

In [15]:
selected_taxi_column.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- flag: string (nullable = true)
 |-- pickupLocationID: string (nullable = true)
 |-- dropoffLocationID: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- extra: string (nullable = true)
 |-- total_amount: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- dropoff_time: string (nullable = true)
 |-- pickup_time: string (nullable = true)



# creating an extra column (trip_year) ,extracting the year character from pickup_time

In [16]:
selected_taxi_column = selected_taxi_column.withColumn('tripYear',year(col('pickup_time')))

selected_taxi_column.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- flag: string (nullable = true)
 |-- pickupLocationID: string (nullable = true)
 |-- dropoffLocationID: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- extra: string (nullable = true)
 |-- total_amount: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- dropoff_time: string (nullable = true)
 |-- pickup_time: string (nullable = true)
 |-- tripYear: integer (nullable = true)



# grouping the data and performing some aggregation on it

In [17]:
yellowTaxiReport = selected_taxi_column.groupBy('pickupLocationID','dropoffLocationID').agg(avg('tripYear').alias('avgTripTimes'),sum('total_amount')).alias('sumTotal').orderBy('pickupLocationID','dropoffLocationID')


yellowTaxiReport.show()

+----------------+-----------------+------------+------------------+
|pickupLocationID|dropoffLocationID|avgTripTimes| sum(total_amount)|
+----------------+-----------------+------------+------------------+
|              10|               10|      2022.0|               4.3|
|              10|              127|      2022.0|             59.35|
|              10|              141|      2022.0|             80.87|
|              10|              163|      2022.0|             67.61|
|              10|              237|      2022.0|              75.3|
|              10|              261|      2022.0|             75.47|
|              10|               42|      2022.0|              76.6|
|             100|              100|      2022.0|             53.76|
|             100|              107|      2022.0|             96.14|
|             100|              113|      2022.0| 97.69999999999999|
|             100|              114|      2022.0|             46.46|
|             100|              11

# performing optimization using persist() to cache data between disk and memory

In [18]:
yellowTaxiReport.persist(pyspark.StorageLevel.MEMORY_AND_DISK)

DataFrame[pickupLocationID: string, dropoffLocationID: string, avgTripTimes: double, sum(total_amount): double]

# we would save to data to both disk and postgre database

In [19]:
(yellowTaxiReport.write.option('header','true')
          .option('date_format','yyyy-MM-dd HH:mm:ss.S')
          .mode('overwrite').partitionBy('pickupLocationID')
          .csv('transformData'))

# writing to database

In [ ]:
db_properties = {}
dburl = database_url
db_properties['user'] =username
db_properties['password']= password


In [ ]:
yellowTaxiDb = yellowTaxiReport.write.jdbc(url= dburl,table='yellowtaxi',mode='overwrite',properties=db_properties)